In [53]:
# Libraries, importing API keys
from binance import Client
import pandas as pd
import sys
import datetime
import numpy as np
from get_api_keys import *
import re
import time
import seaborn as sn
import matplotlib.pyplot as plt

In [54]:
# Options
backperiod='200'
pd.options.display.max_rows = 2100
pd.options.display.max_columns = 10

In [55]:
# Connect to client and pull tickers
client = Client(api_key, api_secret)
info = client.get_ticker()
print("Connected to Binance")

Connected to Binance


In [56]:
# Pass into a dataframe
df = pd.DataFrame(info, columns =['symbol', 'lastPrice', 'volume', 'quoteVolume'])
df=df[df['symbol'].str.contains("USDT")==True]
df['quoteVolume']=df['quoteVolume'].astype(float)
df = df.sort_values('quoteVolume', ascending = False)

# After checking 'df', define coins
coins = ['BTCUSDT', 'ETHUSDT', 'MATICUSDT', 'SOLUSDT',
         'BNBUSDT', 'XRPUSDT', 'ADAUSDT', 'ATOMUSDT', 
         'AVAXUSDT','DOGEUSDT', 'SHIBUSDT','NEARUSDT',
         'DOTUSDT', 'GALAUSDT', 'SANDUSDT', 'LINKUSDT',
         'FTMUSDT', 'LTCUSDT', 'WAVESUSDT', 'MANAUSDT',
         'UNIUSDT', 'AAVEUSDT', 'ALGOUSDT']
# All possible combinations
#res = [(a, b) for idx, a in enumerate(coins) for b in coins[idx + 1:]]
#coins = ['DOTUSDT']
print("Pairs defined")

Pairs defined


In [57]:
# Dataset for each pair
myDataFrame = pd.DataFrame()
list1 = []
for x in coins:
    # Get candlestick data for each coin
    print('Getting data for: '+x)
    klines = pd.DataFrame(client.get_historical_klines(x,
                                                       Client.KLINE_INTERVAL_1HOUR,
                                                       backperiod+" days ago ECT"))
    # Define and select columns
    klines.columns = ["open_time", "Open", "High",
                     "Low", "Close", "Volume",
                     "close_time", "Quote_asset_volume",
                     "N_trades", "Taker_buy_base_asset_volume",
                     "taker_buy_quote_asset_volume", "ignore"]
    klines_exp = klines[["open_time", "close_time", "Open", "Close", "High", "Low", "Volume"]]

    klines_exp.insert(loc=0, column='pair', value=x)
    klines_exp.reset_index(drop=True, inplace=True)
    klines_exp=klines_exp.loc[klines_exp['pair'] == x, 'Close'].to_frame()
#   klines_exp=klines_exp.loc[klines_exp['pair'] == x, ['Close', 'close_time']]
    
    
    
    list1.append(klines_exp)
    time.sleep(1)

print(list1)
list2=pd.concat(list1, axis=1)
print("Downloading complete")



Getting data for: BTCUSDT
Getting data for: ETHUSDT
Getting data for: MATICUSDT
Getting data for: SOLUSDT
Getting data for: BNBUSDT
Getting data for: XRPUSDT
Getting data for: ADAUSDT
Getting data for: ATOMUSDT
Getting data for: AVAXUSDT
Getting data for: DOGEUSDT
Getting data for: SHIBUSDT
Getting data for: NEARUSDT
Getting data for: DOTUSDT
Getting data for: GALAUSDT
Getting data for: SANDUSDT
Getting data for: LINKUSDT
Getting data for: FTMUSDT
Getting data for: LTCUSDT
Getting data for: WAVESUSDT
Getting data for: MANAUSDT
Getting data for: UNIUSDT
Getting data for: AAVEUSDT
Getting data for: ALGOUSDT
[               Close
0     38151.17000000
1     38673.03000000
2     38876.90000000
3     38866.27000000
4     38782.91000000
...              ...
4795  21507.10000000
4796  21542.42000000
4797  21556.42000000
4798  21590.30000000
4799  21413.51000000

[4800 rows x 1 columns],               Close
0     2695.63000000
1     2702.61000000
2     2713.98000000
3     2717.68000000
4     27

In [58]:
# print(list2)
list2.drop(list2.tail(5).index,inplace=True)
list2.drop(list2.head(5).index,inplace=True) # drop first n rows
print(list2)


               Close          Close       Close        Close         Close  \
5     39016.39000000  2740.74000000  1.58400000  92.36000000  382.70000000   
6     38960.24000000  2731.44000000  1.57700000  91.67000000  381.30000000   
7     38660.07000000  2707.45000000  1.55300000  90.66000000  377.90000000   
8     38512.51000000  2688.35000000  1.54300000  89.72000000  378.90000000   
9     38638.24000000  2710.78000000  1.56400000  90.74000000  380.90000000   
...              ...            ...         ...          ...           ...   
4790  21606.07000000  1750.70000000  0.90250000  34.84000000  294.70000000   
4791  21758.00000000  1783.40000000  0.90720000  35.30000000  297.50000000   
4792  21648.34000000  1774.12000000  0.90600000  35.03000000  296.70000000   
4793  21548.91000000  1772.09000000  0.90740000  35.01000000  296.10000000   
4794  21610.85000000  1763.56000000  0.90050000  35.15000000  295.10000000   

      ...        Close       Close       Close         Close   

In [59]:
new_cols = coins
list2.columns = new_cols


list2 = list2.apply(pd.to_numeric)
corrMatrix = list2.corr()

pair1=[]
pair2=[]

for i in corrMatrix.columns:
    for j in corrMatrix.index.values:
        if 0.95 <= corrMatrix.loc[i, j] < 1:
            tmp=i+"_"+j
            pair1.append(i)
            pair2.append(j)

d = {'pair1': pair1, 'pair2': pair2}
df = pd.DataFrame(data=d)

df.to_csv("high_correlated_pairs.csv", index=False)
print('Calculations of correlation finished')

Calculations of correlation finished
